In [ ]:
import pandas as pd
import numpy as np
import glob
import os

def euclidean_distance(p1, p2):
    return np.sqrt((p1[0] - p2[0]) ** 2 + (p1[1] - p2[1]) ** 2)

def match_colors(csv_file, txt_file, output_file):
    # CSVを読み込む
    df_csv = pd.read_csv(csv_file)
    frame_col = df_csv.columns[0]  # frame_numberのカラム名
    
    # 各座標と色の対応を作成
    color_map = {}
    for col in df_csv.columns[1:]:
        if col.startswith("x_"):
            color = col.split('_')[-1]  # 色名のみ取得
            y_col = f"y_{col[2:]}"
            color_map[color] = df_csv[[frame_col, col, y_col]].rename(columns={col: 'x', y_col: 'y'})
    
    # TXTを読み込む
    df_txt = pd.read_csv(txt_file, header=None, names=['frame', 'id', 'x', 'y'])
    df_txt['initial_loc'] = ''  # 新しいカラムを追加
    
    # 各フレームごとに処理
    for frame in df_txt['frame'].unique():
        txt_subset = df_txt[df_txt['frame'] == frame]
        csv_subset = {k: v[v[frame_col] == frame] for k, v in color_map.items()}
        
        for i, row in txt_subset.iterrows():
            min_dist = float('inf')
            best_match = None
            
            for color, df in csv_subset.items():
                if not df.empty:
                    dist = euclidean_distance((row['x'], row['y']), (df.iloc[0]['x'], df.iloc[0]['y']))
                    if dist < min_dist:
                        min_dist = dist
                        best_match = color[2:]
            
            if best_match:
                df_txt.at[i, 'color'] = best_match  # 色名のみを設定
    
    # 出力
    df_txt.to_csv(output_file, index=False, header=False)

# ファイル一覧を取得
csv_files = sorted(glob.glob("../../ground_truth/Indoor/tracking_data_with_color/*.csv"))
txt_files_pre = sorted(glob.glob("../../ground_truth/Indoor/transformed_MOT/*_pre.txt"))
txt_files_post = sorted(glob.glob("../../ground_truth/Indoor/transformed_MOT/*_post.txt"))

txt_files = txt_files_pre + txt_files_post  # preとpostの両方を処理

# ファイル名の対応を取る
for csv_file, txt_file in zip(csv_files, txt_files):
    output_file = txt_file.replace(".txt", "_color.txt")
    match_colors(csv_file, txt_file, output_file)
    print(f"Processed {output_file}")
